In [9]:
#Build a Multilayer Convolutional Network

In [16]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
import tensorflow as tf
tf.reset_default_graph()
sess = tf.InteractiveSession()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [17]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [18]:
def weight_variable(shape):
    initial=tf.truncated_normal(shape=shape,stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
#output[i] = reduce(value[strides * i:strides * i + ksize])

In [19]:
 #first conv layer [m 28*28*1]->[m 28 28 1]->[m 28 28 32]->[m 14 14 32]
W_conv1 = weight_variable([5, 5, 1, 32])#filting 
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])#-1 means that not changed ,[m ,28*28*2]->[m,28,28,1]
h_conv1=tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1=max_pool_2x2(h_conv1)
 #second convolutional layer [m 14 14 32]->[m 14 14 64]->[m 7 7 64]
W_conv2=weight_variable([5,5,32,64])
b_conv2=bias_variable([64])
h_conv2=tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2=max_pool_2x2(h_conv2)
 #full-connected layer [m 7 7 64]->[m 7*7*64]->[m 1024]
h_pool2_flat=tf.reshape(h_pool2,[-1, 7*7*64])
W_fc1=weight_variable([7*7*64,1024])
b_fc1=bias_variable([1024])
h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

In [20]:
#droput reduce overfitting befor readout layer
keep_prob=tf.placeholder(dtype=tf.float32)
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob=keep_prob)
 #readout layer [m 1024]->[m 10]
W_fc2=weight_variable([1024,10])
b_fc2=bias_variable([10])
y_conv=tf.matmul(h_fc1_drop,W_fc2)+b_fc2

In [25]:
 #train and evaluate model
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y_conv))
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction=tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(2000):
    batch=mnist.train.next_batch(50)
    if i%100==0:
        train_accuracy=accuracy.eval(feed_dict={
                x:batch[0],y_:batch[1],keep_prob:1.0})
        print("step %d, training accuracy %g\n"%(i,train_accuracy))
    train_step.run(feed_dict={x:batch[0],y_:batch[1],keep_prob:0.5})
print("test accuracy %g\n"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

ResourceExhaustedError: OOM when allocating tensor with shape[3136,1024]
	 [[Node: Variable_4/Adam_3/Assign = Assign[T=DT_FLOAT, _class=["loc:@Variable_4"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/gpu:0"](Variable_4/Adam_3, Variable_4/Adam_3/Initializer/Const)]]

Caused by op 'Variable_4/Adam_3/Assign', defined at:
  File "/home/keetsky/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/keetsky/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/keetsky/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/keetsky/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/keetsky/.pyenv/versions/anaconda3-4.3.0/lib/p